In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Colab Notebooks/AI+X 중급

/content/drive/MyDrive/Colab Notebooks/AI+X 중급


In [1]:
! pip install gradio
! pip install openai

     --------------------------------------- 20.0/20.0 MB 38.4 MB/s eta 0:00:00
     ---------------------------------------- 50.5/50.5 kB ? eta 0:00:00
     ---------------------------------------- 2.1/2.1 MB 44.9 MB/s eta 0:00:00
     ---------------------------------------- 124.7/124.7 kB ? eta 0:00:00
     -------------------------------------- 151.7/151.7 kB 9.4 MB/s eta 0:00:00
     ---------------------------------------- 58.3/58.3 kB 3.0 MB/s eta 0:00:00
     ---------------------------------------- 236.8/236.8 kB ? eta 0:00:00
     ------------------------------------- 471.5/471.5 kB 30.8 MB/s eta 0:00:00
     ------------------------------------- 288.1/288.1 kB 18.5 MB/s eta 0:00:00
     ---------------------------------------- 45.7/45.7 kB ? eta 0:00:00
     ---------------------------------------- 191.7/191.7 kB ? eta 0:00:00
     ---------------------------------------- 57.1/57.1 kB ? eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): fin


[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
user_name_list = []
with open("user_list.json", 'w') as outfile:
      json.dump(user_name_list, outfile)

In [26]:
import openai
import gradio as gr
import json
from datetime import date

# 저장해둔 api key 불러옴
key_file = open("chatgpt_api_key.txt", "r")
key_data = key_file.read()
key_file.close()  

# 발급받은 API 키 설정
OPENAI_API_KEY = key_data

# openai API 키 인증
openai.api_key = OPENAI_API_KEY

# 직접 날짜 받아올때 사용
# date = date.today()
# today = date.isoformat()
# today
today = "2023-01-08"
today

# 나중에 user_name은 웹에서 받아오기
user_name = "Amanda" 
file_name = user_name + "_history.json"

with open("user_list.json", 'r') as f:   
    user_name_list = json.load(f)

message_history = []

if user_name in user_name_list:
  with open(file_name, 'r') as f:   
    message_history = json.load(f)
else:
  user_name_list.append(user_name)

with open("user_list.json", 'w') as outfile:
      json.dump(user_name_list, outfile)

message_history.append({"role": "system", "content": f"When you reply, user's workout is done."})
message_history.append({"role": "system", "content": f"You are specialized in workout motivation, and you will reply very short."})     
message_history.append({"role": "system", "content": f"Your name is DZM"})         
message_history.append({"role": "system", "content": f"my name is "+f"{user_name}"})
message_history.append({"role": "system", "content": "Today is " + f"{today}." + " Please remember I worked out today. The days I didn't say were the days I didn't exercise."})
message_history.append({"role": "system", "content": f"The existing date will be maintained until a new date is entered."})         
init_len = len(message_history)

# chatbot 구현
def chat_model(input):

    message_history.append({"role": "user", "content": f"{input}"})

    chat = openai.ChatCompletion.create(
            model = "gpt-3.5-turbo", 
            messages = message_history,
        )

    reply_content = chat.choices[0].message.content
    message_history.append({"role": "assistant", "content": f"{reply_content}"})

    with open(file_name, 'w') as outfile:
      json.dump(message_history, outfile)

    response = [(message_history[i]["content"], message_history[i+1]["content"]) for i in range(init_len, len(message_history)-1, 2)]
    
    return response

# gradio
with gr.Blocks() as demo: 

    chatbot = gr.Chatbot() 

    with gr.Row(): 
        txt = gr.Textbox(show_label=False, placeholder="Enter text and press enter").style(container=False)

    txt.submit(chat_model, txt, chatbot)
 
    txt.submit(None, None, txt, _js="() => {''}") 

gr.Interface(fn=chat_model, inputs=txt, outputs=chatbot, title="DZM",
             description="How was your workout today?",
             theme="compact").launch(share=True)

/usr/local/lib/python3.10/dist-packages/gradio/blocks.py:666: UserWarning: Cannot load compact. Caught Exception: The space compact does not exist
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://7ca13cb837c8d2f24a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
